<a href="https://colab.research.google.com/github/zyang63/cost_modeling/blob/main/Feeders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geometric Analysis in Google Colab
This is a code that can analyze STL files for riser locations using voxel meshing and 3D image processing tools. The following cell loads four libraries, trimesh, matplotlib, SimpleITK, and scipy. Trimesh is a python library for meshing which can load stl files and create a voxel mesh which can be analyzed in SimpleITK. Matplotlib is used for visualization, and scipy for matrix maniputlation and image morphological transformations. The cell below loads all of the libraries without output.

**NOTE: Running this the first time in the Google environment will take time but subsequent runs will be faster.**

In [ ]:
%%capture
!pip install trimesh
!pip install matplotlib
!pip install SimpleITK
!pip install scipy
!pip install numpy-stl
!pip install cv2

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SA30053 3D MODEL (2).stl to SA30053 3D MODEL (2).stl


# Voxelizing the geometry
Load the geometry and analyze for the size of the geometry. Set the # of elements for creating the voxel mesh.

In [ ]:
import numpy as np
import trimesh as t
from trimesh.voxel import creation

In [ ]:
# import filepath
str_filePath = list(uploaded.keys())[0]
geometry = t.load_mesh(str_filePath)

## Setting the element count
This code sets the element count based on the number of divisions along the maximum length. In this case we are setting 200 divisions along the maximum length. This means that the 2 other dimensions are smaller and will have less elements. The size of the element is reported based on the maximum length found in the geometry (regardless of units) divided by the number of divisions on that length. The example below shows us that if the length is in mm and the number of divisions is 200 that the element size is 2.25mm/cell.

In [ ]:
element_count = 200
voxel_size = geometry.extents.max()/element_count
print(voxel_size)

5.535


## Meshing
Creating the mesh using trimesh. The mesh only sets a 1 for the boundary elements and not inside or outside the stl file. therefore the inside of the mesh needs to be set to 1 for the distance transformation later. Padding is necessary because the geometry touches the boundary and we want the outside geometry to be connected.

**Can we do the filling here? To simplify the readability of the code**

In [ ]:
voxel_surface = t.voxel.creation.voxelize(mesh = geometry, pitch = voxel_size)
print(voxel_surface)
array_int = voxel_surface.matrix.astype(int)
array_pad = np.pad(array_int,((1,1)),'constant')


<trimesh.VoxelGrid(201, 144, 63)>


# Morphological operations, Distance Field, Watershed on mesh
As mentioned the geometry as meshed does not set the internal cells only the surface. Thus, the code block below fills the geometry so that further morphological operations can be done. The distance field is just a signed distance field generated using the SimpleITK SingedMaurerDistance function. Maurer et al. created a linear time algorithm for calculated the exact distance field and so is the fastest method for this to date. The watershed algorithm is a morphological operation using the distance field to segment the geometry from the deepest areas of the distance field solution. Typically without filtering there are many small segments near the edge due to mesh irregularities without the initial erosion/dilation step. In addition, there is a upper limit to the number of possible segments based on the number of triangles in the polyhedral geometry that the stl provides. So with finer meshes of an stl geometry expect more isolated segments. This observation works both ways so while it may not be desirable to have moany segments, there is also an upper limit to the total number of possible segments in a given stl geometry thus further refinement is not improving the solution quality.  Increasing the smoothing by erosion/dilation is expected to reduce geometric accuracy at sharp features therefore other strategies will be used to combine watershed segments.

In [ ]:
import SimpleITK as sitk
import numpy as np
from scipy import ndimage
img = sitk.GetImageFromArray(array_pad)
seg = sitk.ConnectedComponent(img != img[0,0,0])
img_filled = sitk.BinaryFillhole(seg!=0)
array_filled = sitk.GetArrayFromImage(img_filled)
conndef = ndimage.generate_binary_structure(3, 1)

## Erosion and Dilation
Instead of filtering the distance field for rough surfaces caused by the numerical operation of creating the distance field from a voxel mesh, this can be smoothed but erosion and dilation of the mesh. The erosion function removes the layers of the geometry by the connectivity structure element generally one layer at a time. Here 1 layer is being removed to begin the distance function within the surface instead of at or ouside the.

In [ ]:
array_erosion = ndimage.binary_erosion(array_filled, structure=conndef, iterations=2, mask=None,  border_value=0, origin=0, brute_force=False)
array_dilate = ndimage.binary_dilation(array_erosion, structure=conndef, iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
array_erosion_int = array_dilate.astype(int)
img_erosion = sitk.GetImageFromArray(array_erosion_int)

## Distance Field Calculation
This is calculating the distance field for the smoothed geometry prepared from the STL file. Need a reference for the Maurer map this is chosen because its linear time for computing the distance field. Also the other choices for the function should be described briefly including the "insideIsPositive", "squaredDistance" and "useImageSpacing". Typically, it's also useful to describe the input data "img_erosion" and the output "img_dist".

In [ ]:
img_dist = sitk.SignedMaurerDistanceMap(img_erosion != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False)
array_dist = sitk.GetArrayFromImage(img_dist)
array_dist_inside = array_dist
array_dist_inside[array_erosion_int != 1] = 0

## Watersheds
A watershed is a method of image segmentation that partitions an image into regions or segments based on the image intensity or gradient. The goal of watershed segmentation is to separate regions with different intensity or gradient values that correspond to different objects or features in an image.


In [ ]:
img_thresh = img_filled != 0
img_ws = sitk.MorphologicalWatershed( img_dist, markWatershedLine=True, level=1)
img_ws_mark = sitk.Mask( img_ws, sitk.Cast(img_thresh, img_ws.GetPixelID()))
array_ws_mark = sitk.GetArrayFromImage(img_ws_mark)

# Feeder Analysis Section
As discussed above there will likely be more segments for feeder evaluation than is typical in a sand casting process. This is due to the relatively low value of the Bi number associated with a low heat transfer at the surface compared to the thermal conductivity in the casting. This leads to significant diffusion of the thermal hotspots merging geometric centers. Even though this can be predicted with a high degree of accuracy using a real physical model of heat transfer, it is the objective of this work to decouple the geometric effect from the physical property effect. Thus solutions for merging hotspots is sought that do not depend on the properties of the material and instead anticipate geometric correlations that are relevant to the processing conditions. Warriner proposed an adjaceny graph of the segments reducing the information of each to a depth and separation distance. This information can be used to weight the edges of the graph such that each nodal contribution can be ranked and then the connected regions can be grouped using a greedy algorithm. Greedy in the sense that the biggest node then consumes the neighboring ones and then they are taken off the list such that the solution can depend on the node which is chosen at the start. In practice, this method tends to provide asymetric segments which actually is an advanged in creating risers in symmetric parts.

## Feeder parameter E calculated
$$E_{ij} = \frac{k(m_i+m_j)}{d_{ij}}$$

$i$ and $j$ be two connected segments
$m$ is value of Euclidean distance transform
$d_{ij}$ is the Euclidean distance between two hotspots

In [ ]:
# Find coordinates of the minimum value in distance field
def find_coordinates(array_dist):
    indices = np.nonzero(array_dist == array_dist.min())
    if indices[0].size == 0:
          return None
    return indices[0][0], indices[1][0], indices[2][0]

### Computed parameter E for all connected segments in the given geometry

In [ ]:
E_list = []
G = array_ws_mark.max() + 1
# Loop through all pairs of bounding boxes and check if they intersect
for i in range(1, G):
  F_i = np.where(array_ws_mark != i, 0, array_ws_mark)
  dilated_array_i = ndimage.binary_dilation(F_i, structure=conndef).astype(F_i.dtype)
  T_i = np.where(F_i != i, 0, array_dist_inside)
  M_i = T_i.min()
  coordinates_i = find_coordinates(T_i)
  x_i, y_i, z_i = coordinates_i
  for j  in range(i+1, G):
          F_j = np.where(array_ws_mark != j, 0, array_ws_mark)
          dilated_array_j = ndimage.binary_dilation(F_j, structure=conndef).astype(F_j.dtype)
          Union =  dilated_array_i + dilated_array_j
          if 2 in Union:
            T_j = np.where(F_j != j, 0, array_dist_inside)
            M_j = T_j.min()
            coordinates_j = find_coordinates(T_j)
            x_j, y_j, z_j = coordinates_j
            # Calculate the distance between i and j using the Euclidean distance formula
            E = 1.7*(abs(M_i) + abs(M_j))/np.sqrt((x_i - x_j)**2 + (y_i - y_j)**2 + (z_i - z_j)**2)
            E_list.append((i, j, E))
print(E_list)

[(1, 2, 0.16248836941189237)]


### Feeder edge cutting

In [ ]:
E_list_cut = [tup for tup in E_list if tup[2] >= 0]
print(E_list_cut)

[(1, 2, 0.16248836941189237)]


## Feeder parameter N calculated
$$N_i = m_i \cdot \left(1 + \sum_j E_{ij}\right)$$

In [ ]:
N_list = []
for i in range(1, G):
    F_i = np.where(array_ws_mark != i, 0, array_ws_mark)
    T_i = np.where(F_i != i, 0, array_dist_inside)
    M_i = T_i.min()
    total_weight = 1
    for edge in E_list:
        if edge[0] == i or edge[1] == i:
            total_weight += edge[2]
    N = abs(M_i) * ( total_weight * float(voxel_size))
    N_list.append((N,i))
print(N_list)

[(55.35057161749578, 1), (55.35057161749578, 2)]


## Feeder merging calculated
The resulting node influence values were sorted in descending order. The sorted list was then looped over, and at each iteration, the current node and all of its adjacent neighbors were assigned to a cluster. This assignment was performed in a greedy manner, such that once a node was assigned to a cluster, it was no longer considered for future assignment. Additionally, any edges linking nodes of different clusters were removed during the clustering process.

In [ ]:
sorted_N_list = sorted(N_list, reverse=True, key=lambda x: x[0])
second_N_list_arr = [x[1] for x in sorted_N_list]
print(second_N_list_arr)
connected_arr = [(x[0], x[1]) for x in E_list_cut]
print(connected_arr)
Connected = []
for elem in second_N_list_arr:
  output_lst = list(set([x[0] for x in connected_arr if elem in x] + [x[1] for x in connected_arr if elem in x]))
  Connected.append(output_lst)
seen = set()
Connected_U = []
for a in Connected:
    # Check if the current array has already been seen
    if tuple(a) not in seen:
        seen.add(tuple(a))
        Connected_U.append(a)
print(Connected_U)
result = []

for arr in Connected_U:
    common_elements = []
    for elem in arr:
        if elem in second_N_list_arr:
            common_elements.append(elem)
            second_N_list_arr.remove(elem)
    if common_elements:
        result.append(common_elements)

    # remove empty array from array
    if not arr:
        Connected_U.remove(arr)
# Find the missing elements
missing = set(second_N_list_arr) - set(item for sublist in result for item in sublist)
# Add missing elements to arr1 separately
for element in missing:
    result.append([element])
# Sort result_1 in ascending order
result.sort()
# Print the result
print(result)
num_arrays = len([x for x in result if isinstance(x, list)])
print(num_arrays)

[1, 2]
[(1, 2)]
[[1, 2]]
[[1, 2]]
1


## Feeder parameter calculated and export
To calculate a feeder's geometry based on connected segments volume, parameters such as length (L), width (W), and thickness (T) of the segment, as well as the maximum Euclidean Distance Transform (EDT) value of the segment and geodesic distances from the hotspot position to surface elements are considered. The loaction of largest EDT value determines the length (L), while the median geodesic distance determines the width (W).

In [ ]:
import statistics
import math
stats = sitk.LabelShapeStatisticsImageFilter()
stats.Execute(img_ws_mark)
for p in range(0, num_arrays):
    F = sitk.GetArrayFromImage(img_ws_mark)
    F[np.where(~np.isin(F, result[p]))] = 0
    T = sitk.GetArrayFromImage(img_dist)
    T[array_erosion_int != 1] = 0
    T[~np.isin(F, result[p])] = 0
    # Find coordinates of the minimum value in T
    coordinates = find_coordinates(T)
    x, y, z = coordinates
    # Find first and last non-zero elements in F
    indices = np.where(np.isin(F, result[p]))
    if indices[0].size > 0:
        x1, y1, z1 = indices[0][0], indices[1][0], indices[2][0]
        x2, y2, z2 = indices[0][-1], indices[1][-1], indices[2][-1]
    dis_x = (x2 - x1) * float(voxel_size)
    dis_y = (y2 - y1) * float(voxel_size)
    dis_z = (z2 - z1) * float(voxel_size)
    L = max(dis_x, dis_y, dis_z)
    W = statistics.median([dis_x, dis_y, dis_z])
    T = abs(T.min()) * voxel_size
    shape_factors = (L + W) / T
    bounds = geometry.bounds
    feeders_position = (float(voxel_size) * np.array([x, y, z])) + bounds[0]
    num_elements = len(result[p])
    total_element_count = 0
    for q in range(0, num_elements):
        e = result[p][q]
        element_count = stats.GetNumberOfPixels(e)
        total_element_count += element_count
        #print(total_element_count)
    segment_volumes = float(total_element_count)*(float(voxel_size)**3)
        #print(segment_volumes)
    feeders_volume = 2.51 * segment_volumes * (shape_factors ** -0.74)
    feeders_radius = (feeders_volume / (3 * math.pi)) ** (1 / 3)
    feeders_height = 3 * feeders_radius + T
    #print(feeders_height)
    cylinder = t.creation.cylinder(radius=feeders_radius, height=feeders_height, sections=50)
    cylinder.apply_translation(feeders_position + [0, 0, feeders_height/2])
    cylinder.export(f'feeder{p}.stl')

# Visualization output

## Blender installed

In [ ]:
import os

def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(os.path.relpath(root,start = os.curdir), name)
!wget https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.4/blender-3.4.1-linux-x64.tar.xz -O BlenderDownload.tar.xz
!tar xf /content/BlenderDownload.tar.xz
blender_path = find('blender', '/content/')
print(blender_path)

--2023-07-19 02:53:10--  https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.4/blender-3.4.1-linux-x64.tar.xz
Resolving ftp.nluug.nl (ftp.nluug.nl)... 145.220.21.40, 2001:67c:6ec:221:145:220:21:40
Connecting to ftp.nluug.nl (ftp.nluug.nl)|145.220.21.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223772600 (213M) [application/x-tar]
Saving to: ‘BlenderDownload.tar.xz’

BlenderDownload.tar 100%[===================>] 213.41M  30.8MB/s    in 12s     

2023-07-19 02:53:22 (17.7 MB/s) - ‘BlenderDownload.tar.xz’ saved [223772600/223772600]

blender-3.4.1-linux-x64/blender


## Export visualized colored feeders GLB file

### Blender Python code which function included geometry transparented and feeders colored

In [ ]:
data = [
    'import bpy',
    'bpy.ops.object.select_all(action="SELECT")',
    'bpy.data.objects["Camera"].select_set(False)',
    'bpy.data.objects["Light"].select_set(False)',
    'bpy.ops.object.delete()',
    # Import file path
    f'bpy.ops.import_mesh.stl(filepath="//content//{str_filePath}")',
    'obj = bpy.context.object',
    'obj.color = (0,0,1,1)',
    'mat = bpy.data.materials.new("0")',
    'mat.use_nodes = True',
    'principled = mat.node_tree.nodes["Principled BSDF"]',
    'principled.inputs["Base Color"].default_value = (1,1,1,1)',
    'principled.inputs["Alpha"].default_value = 0.7',
    'obj.data.materials.append(mat)',
    'bpy.context.object.active_material.blend_method = "BLEND"',
]

for i in range(0, num_arrays):
    data += [
        f'bpy.ops.import_mesh.stl(filepath="//content//feeder{i}.stl")',

        'obj = bpy.context.object',
        'obj.color = (0,0,1,1)',
        f'mat = bpy.data.materials.new("{i+1}")',
        'mat.use_nodes = True',
        'principled = mat.node_tree.nodes["Principled BSDF"]',
        'principled.inputs["Base Color"].default_value = (0,0,1,1)',
        'obj.data.materials.append(mat)',]

        # Output file path
data += [
'bpy.ops.export_scene.gltf(filepath="//content//colored_geometry.glb")'
    ]

with open('colored_geometry.py', 'w') as f:
    f.write('\n'.join(data))

In [ ]:
!./$blender_path -b  -noaudio -P "/content/colored_geometry.py"

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Import finished in 0.0039 sec.
Import finished in 0.0009 sec.
02:53:50 | INFO: Draco mesh compression is available, use library at /content/blender-3.4.1-linux-x64/3.4/python/lib/python3.10/site-packages/libextern_draco.so
02:53:51 | INFO: Starting glTF 2.0 export
02:53:51 | INFO: Extracting primitive: SA30053 3D MODEL (2)
02:53:51 | INFO: Primitives created: 1
02:53:51 | INFO: Extracting primitive: feeder0
02:53:51 | INFO: Primitives created: 1
02:53:51 | INFO: Finished glTF 2.0 export in 0.013018608093261719 s


Blender quit


In [ ]:
script = f"""
import bpy

# Delete all objects except for the camera and light
bpy.ops.object.select_all(action='SELECT')
bpy.data.objects['Camera'].select_set(False)
bpy.data.objects['Light'].select_set(False)
bpy.ops.object.delete()

# Import STL mesh and apply remesh modifier
bpy.ops.import_mesh.stl(filepath="/content/{str_filePath}",global_scale=0.1)
bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = {voxel_size}/10
bpy.ops.object.modifier_apply(modifier="Remesh")

# Unwrap the mesh using Smart UV Project
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')

# Export the object as an OBJ file
for obj in bpy.context.selected_objects:
    if obj.type == "MESH":
        bpy.ops.export_scene.obj(filepath="/content/remeshed_uv.obj", use_triangles=True, use_materials=False,global_scale=10)
"""

# Save the script to a text file
with open('obj-export.py', 'w') as f:
    f.write(script)

In [ ]:
!./$blender_path -b  -noaudio -P "/content/obj-export.py"

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Import finished in 0.0040 sec.
    (  0.0002 sec |   0.0000 sec) OBJ Export path: '/content/remeshed_uv.obj'
Error: Object does not have geometry data
          (  0.0005 sec |   0.0001 sec) Finished writing geometry of 'Light'.
Error: Object does not have geometry data
          (  0.0005 sec |   0.0000 sec) Finished writing geometry of 'Camera'.
          (  8.0094 sec |   8.0088 sec) Finished writing geometry of 'SA30053 3D MODEL (2)'.
      (  8.0095 sec |   8.0093 sec) Finished exporting geometry, now exporting materials
      (  8.0096 sec |   8.0093 sec) OBJ Export Finished
Progress: 100.00%


Blender quit


# Generating the surface texture PNG file

The STL risers as well as the remeshed OBJ files have been generated; It is time to generate a texture for the surface of the OBJ file.

## Bulk of the Code
This is the code that performs the UV-Map operations required to change the color of the texture at the desired locations.

### Math and Vectors
Important math classes that will be used by the code

In [ ]:
import numpy as np
import math
import cv2
from stl import mesh

In [ ]:
class Vector2:
    def __init__(self, double_X, double_Y):
        self.x = double_X
        self.y = double_Y

    def __repr__(self):
        return f'Vector2({self.x}, {self.y})'


def toVector2Array(array_str_X, array_str_Y):

    # Get lengths of these arrays
    int_len_X = len(array_str_X)
    int_len_Y = len(array_str_Y)

    # Get minimum one
    int_len_min = int_len_X
    if int_len_Y < int_len_min:
        int_len_min = int_len_Y

    array_int_vector2 = np.empty([int_len_min, 2], dtype=object)
    array_int_vector2[:, :] = [0, Vector2(0, 0)]

    # Iterate from minimum to maximum
    for k in range(int_len_min):
        # Fetch current values
        str_x = array_str_X[k]
        str_y = array_str_Y[k]

        # Convert to double
        double_x = float(str_x)
        double_y = float(str_y)

        # Build vector
        vector2_built = Vector2(double_x, double_y)

        # Add to array
        array_int_vector2[k][0] = k
        array_int_vector2[k][1] = vector2_built

        # print('Vec #', k, ' ~ ', double_x, ' ~ ', double_y)

    return array_int_vector2

In [ ]:
class Vector3:
    def __init__(self, double_X, double_Y, double_Z):
        self.x = double_X
        self.y = double_Y
        self.z = double_Z

    def __repr__(self):
        return f'Vector3({self.x}, {self.y}, {self.z})'

    def toUVSpace(self):

        # Blender might export the OBJ file with different axes than the STL file
        return Vector3(self.x, self.z, -self.y)

    def toSTLSpace(self):

        # This should reverse 'toUVSpace' operation
        return Vector3(self.x, -self.z, self.y)


def toVector3Array(array_str_X, array_str_Y, array_str_Z):
    # Get lengths of these arrays
    int_len_X = len(array_str_X)
    int_len_Y = len(array_str_Y)
    int_len_Z = len(array_str_Z)

    # Get minimum one
    int_len_min = int_len_X
    if int_len_Y < int_len_min:
        int_len_min = int_len_Y
    if int_len_Z < int_len_min:
        int_len_min = int_len_Z

    array_int_vector3 = np.empty([int_len_min, 2], dtype=object)
    array_int_vector3[:, :] = [0, Vector3(0, 0, 0)]

    # Iterate from minimum to maximum
    for k in range(int_len_min):
        # Fetch current values
        str_x = array_str_X[k]
        str_y = array_str_Y[k]
        str_z = array_str_Z[k]

        # Convert to double
        double_x = float(str_x)
        double_y = float(str_y)
        double_z = float(str_z)

        # Build vector
        vector3_built = Vector3(double_x, double_y, double_z)

        # Add to array
        array_int_vector3[k][0] = k
        array_int_vector3[k][1] = vector3_built

        # print('Vec #', k, ' ~ ', double_x, ' ~ ', double_y, ' ~ ', double_z)

    return array_int_vector3

In [ ]:
def getMajorBound(uvFace):
    if len(uvFace.vertices) == 0:
        return Vector3(0, 0, 0)

    double_x = uvFace.vertices[0].v.x
    double_y = uvFace.vertices[0].v.y
    double_z = uvFace.vertices[0].v.z

    # Through all the vertices
    for uvVertex_vertex in uvFace.vertices:
        vector3_v = uvVertex_vertex.v

        # Greater than any?
        if vector3_v.x > double_x:
            double_x = vector3_v.x

        # Greater than any?
        if vector3_v.y > double_y:
            double_y = vector3_v.y

        # Greater than any?
        if vector3_v.z > double_z:
            double_z = vector3_v.z

    # Fill info
    vector3 = Vector3(double_x, double_y, double_z)
    return vector3

In [ ]:
def getMinorBound(uvFace):
    if len(uvFace.vertices) == 0:
        return Vector3(0, 0, 0)

    double_x = uvFace.vertices[0].v.x
    double_y = uvFace.vertices[0].v.y
    double_z = uvFace.vertices[0].v.z

    # Through all the vertices
    for uvVertex_vertex in uvFace.vertices:
        vector3_v = uvVertex_vertex.v

        # Lower than any?
        if vector3_v.x < double_x:
            double_x = vector3_v.x

        # Lower than any?
        if vector3_v.y < double_y:
            double_y = vector3_v.y

        # Lower than any?
        if vector3_v.z < double_z:
            double_z = vector3_v.z

    # Fill info
    vector3 = Vector3(double_x, double_y, double_z)
    return vector3

### File Readers
These files read .OBJ, .VTK, and a variety of image files

In [ ]:
class UVVertex:
    def __init__(self, vector3_v, vector2_vt):
        self.v = vector3_v
        self.vt = vector2_vt

    def __repr__(self):
        return f'UVVertex({repr(self.v)}, {repr(self.vt)})'

In [ ]:
class UVFace:
    def __init__(self, array_uvVertex_vertices, int_faceIndex):
        self.vertices = array_uvVertex_vertices
        self.box_min = getMinorBound(self)
        self.box_max = getMajorBound(self)
        self.adjacent = np.empty(1)
        self.index = int_faceIndex

    def __repr__(self):
        if len(self.vertices) < 1:
            return 'empty_UVFace()'

        # Flatten the matrix and convert elements to strings
        array_flatten = self.vertices.flatten().astype(str)

        # Join the string elements using a separator
        string_result = ', '.join(array_flatten)

        # That should work
        return f'UVFace(np.array([{string_result}]), {self.index})'

    def resolution(self, int_rows, int_columns):

        # Legend says these UVFaces are all 1mm² such that their resolution is
        # equal to the number of pixels of their area. This method actually
        # returns the number of pixels inside this UVFace

        # This faces' bounding box indeed envelops the point.
        uvVertex_U = self.vertices[0]
        uvVertex_V = self.vertices[1]
        uvVertex_W = self.vertices[2]

        # Find 2D Vertices
        vector2_vtU = uvVertex_U.vt
        vector2_vtV = uvVertex_V.vt
        vector2_vtW = uvVertex_W.vt

        # Identify components
        Ux = vector2_vtU.x
        Uy = vector2_vtU.y
        Vx = vector2_vtV.x
        Vy = vector2_vtV.y
        Wx = vector2_vtW.x
        Wy = vector2_vtW.y

        # print('U=({},{}) V=({},{}) W=({},{})'.format(Ux, Uy, Vx, Vy, Wx, Wy))

        # Scale to the image sizes
        Ux = Ux * int_columns
        Vx = Vx * int_columns
        Wx = Wx * int_columns

        Uy = (1 - Uy) * int_rows
        Vy = (1 - Vy) * int_rows
        Wy = (1 - Wy) * int_rows

        Ax = 0
        Ay = 0
        Bx = 0
        By = 0
        Cx = 0
        Cy = 0

        # Identify order in X coordinate
        if Ux <= Vx:
            if Ux <= Wx:
                if Vx <= Wx:
                    # U V W
                    Ax = Ux
                    Ay = Uy

                    Bx = Vx
                    By = Vy

                    Cx = Wx
                    Cy = Wy
                else:
                    # U W V
                    Ax = Ux
                    Ay = Uy

                    Bx = Wx
                    By = Wy

                    Cx = Vx
                    Cy = Vy

            # W comes before U
            else:
                # W U V
                Ax = Wx
                Ay = Wy

                Bx = Ux
                By = Uy

                Cx = Vx
                Cy = Vy

        # V comes before U
        else:
            if Vx <= Wx:
                if Ux <= Wx:
                    # V U W
                    Ax = Vx
                    Ay = Vy

                    Bx = Ux
                    By = Uy

                    Cx = Wx
                    Cy = Wy
                else:
                    # V W U
                    Ax = Vx
                    Ay = Vy

                    Bx = Wx
                    By = Wy

                    Cx = Ux
                    Cy = Uy

            # W comes before V
            else:
                # W V U
                Ax = Wx
                Ay = Wy

                Bx = Vx
                By = Vy

                Cx = Ux
                Cy = Uy

        # print('A=({},{}) B=({},{}) C=({},{})'.format(Ax, Ay, Bx, By, Cx, Cy))
        # print('Scaled A=({},{}) B=({},{}) C=({},{})'.format(Ax, Ay, Bx + 0.00001, By, Cx + 0.00002, Cy))

        # Make sure they are never on the same X
        if Ax == Bx:
            Bx += 0.00001
        if Ax == Cx:
            Cx += 0.00002
        if Bx == Cx:
            Cx += 0.00001

        # Find slopes and intercepts
        mAB = (By - Ay) / (Bx - Ax)
        mAC = (Cy - Ay) / (Cx - Ax)
        mBC = (Cy - By) / (Cx - Bx)

        bAB = Ay - (mAB * Ax)
        bAC = Ay - (mAC * Ax)
        bBC = By - (mBC * Bx)

        # print('Line AB: [m={}, b={}], AC: [m={}, b={}], BC: [m={}, b={}]'.format(mAB, bAB, mAC, bAC, mBC, bBC))

        # Integrate Area
        double_area = 0

        # If these slopes are equal, the three points are a line
        if mAB == mBC:

            # Area of their triangle = 0
            double_area = 0

        else:
            # The relation between the slopes decides the integral
            if mAB > mBC:

                # The triangle has its point upward
                double_area = (0.5 * ((Bx * Bx) - (Ax * Ax)) * (mAB - mAC)) + \
                              ((Bx - Ax) * (bAB - bAC)) + \
                              (0.5 * ((Cx * Cx) - (Bx * Bx)) * (mBC - mAC)) + \
                              ((Cx - Bx) * (bBC - bAC))
            else:

                # The triangle has its point downward
                double_area = (0.5 * ((Bx * Bx) - (Ax * Ax)) * (mAC - mAB)) + \
                              ((Bx - Ax) * (bAC - bAB)) + \
                              (0.5 * ((Cx * Cx) - (Bx * Bx)) * (mAC - mBC)) + \
                              ((Cx - Bx) * (bAC - bBC))

        # Result
        return double_area


def empty_UVFace():
    return UVFace(np.empty(0, dtype=object), -2)


def parse_UVFace(str_indices, array_int_vector3_Vertices, array_int_vector2_TextureVert, int_index):

    # Split by whitespaces
    array_str_indices = str_indices.split()

    # Prepare faces, by default size three
    array_uvVertex_vertices = np.empty(len(array_str_indices), dtype=object)

    for k in range(len(array_str_indices)):

        # Split current line by spaces
        str_line = array_str_indices[k]
        array_str_line = str_line.split('/')

        # First index, V
        if len(array_str_line) < 1:
            print('Invalid Face Vertex ', str_line)
            continue

        # Parse string to double yeah
        double_v = float(array_str_line[0])

        # Round
        int_v = round(double_v)

        # print('V: ~  ', int_v)
        vector3_v = array_int_vector3_Vertices[int_v - 1, 1]

        # First index, VT
        vector2_vt = Vector2(0, 0)
        if len(array_str_line) >= 2:
            # Parse string to double yeah
            double_vt = float(array_str_line[1])

            # Round
            int_vt = round(double_vt)

            # print('VT: ~  ', int_vt)
            vector2_vt = array_int_vector2_TextureVert[int_vt - 1, 1]

        # Include new vertex
        uvVertex_vertex = UVVertex(vector3_v, vector2_vt)
        array_uvVertex_vertices[k] = uvVertex_vertex

    # Initialize Adjacent
    return UVFace(array_uvVertex_vertices, int_index)

In [ ]:
class UVMap:
    def __init__(self, array_uvFace_faces):

        # An UVMap, in the end, is just a glorified array of UVFaces
        self.faces = array_uvFace_faces

    def average_resolution(self, int_rows, int_columns, int_samples):
        total_resolution = 0
        num_faces = 0
        int_total = len(self.faces)
        int_faceIndex = 0

        # Increase the face index
        int_increase = math.floor(int_total / int_samples)
        if int_increase < 1:
            int_increase = 1

        # Sum the resolution of these samples
        while int_faceIndex < int_total:

            # Identify face
            uvFace_face = self.faces[int_faceIndex]
            int_faceIndex += int_increase

            # This face has been counted
            num_faces += 1

            # Double because these are triangles and we want the
            # resolution of the squares they were part of
            total_resolution += 2 * uvFace_face.resolution(int_rows, int_columns)

        # Return the average
        return total_resolution / num_faces

    def corneredBoxFaces(self, vector3_minPoint, vector3_maxPoint, double_tolerance):
        # The point of this function is to map 3D space points to faces in 2D space
        # by first mapping faces to their bounding boxes.

        array_int_uvFace = np.empty([0, 2], dtype=object)

        # Extrema switcheroo
        mx = vector3_minPoint.x if vector3_minPoint.x < vector3_maxPoint.x else vector3_maxPoint.x
        my = vector3_minPoint.y if vector3_minPoint.y < vector3_maxPoint.y else vector3_maxPoint.y
        mz = vector3_minPoint.z if vector3_minPoint.z < vector3_maxPoint.z else vector3_maxPoint.z
        Mx = vector3_minPoint.x if vector3_minPoint.x > vector3_maxPoint.x else vector3_maxPoint.x
        My = vector3_minPoint.y if vector3_minPoint.y > vector3_maxPoint.y else vector3_maxPoint.y
        Mz = vector3_minPoint.z if vector3_minPoint.z > vector3_maxPoint.z else vector3_maxPoint.z

        # Iterate through every uvFace
        int_face_current = 0
        int_face_length = len(self.faces)
        for int_faceIndex in range(int_face_length):

            # Target face
            uvFace_observed = self.faces[int_faceIndex]

            # Find bounding box
            vector3_maxFace = uvFace_observed.box_max
            vector3_minFace = uvFace_observed.box_min

            # Check if the point is within the tolerance of the bounding box
            if (vector3_maxFace.x + double_tolerance) < mx:
                continue
            if (vector3_maxFace.y + double_tolerance) < my:
                continue
            if (vector3_maxFace.z + double_tolerance) < mz:
                continue
            if (vector3_minFace.x - double_tolerance) > Mx:
                continue
            if (vector3_minFace.y - double_tolerance) > My:
                continue
            if (vector3_minFace.z - double_tolerance) > Mz:
                continue

            # This face's bounding box envelops the point.
            int_face_current += 1
            array_int_uvFace = np.append(array_int_uvFace, [[int_faceIndex, uvFace_observed]], axis=0)

        print(f'Cornered Box Faces {array_int_uvFace.shape[0]}/{int_face_length}')
        return array_int_uvFace

def readOBJFile(str_path):

    # Read file, split by newline
    with open(str_path, 'r') as f:
        str_obj_encoded = f.read()
        array_str_obj_encoded = str_obj_encoded.splitlines()

    # Count final Vertex and Texture array sizes
    int_vertex_count = 0
    int_texture_count = 0
    int_faces_count = 0
    for k in range(len(array_str_obj_encoded)):

        # Split current line by spaces
        str_line = array_str_obj_encoded[k]
        array_str_line = str_line.split()

        # Which is its identifier key?
        str_ident = array_str_line[0]
        if str_ident == 'v':

            # Key 'v' ~ Vertex position, requires X Y Z
            if len(array_str_line) < 4:
                print(f'Invalid Vertex {str_line}')
                continue

            int_vertex_count += 1

        elif str_ident == 'vt':

            # Key 'vt' ~ Texture Coordinates, requires U V
            if len(array_str_line) < 3:
                print(f'Invalid Tex UV {str_line}')
                continue

            int_texture_count += 1

        elif str_ident == 'f':

            # Key 'f' ~ Face information, requires one entry at least
            if len(array_str_line) < 2:
                print(f'Invalid Face {str_line}')
                continue

            int_faces_count += 1

    # print('Counter Vertices: x' + str(int_vertex_count))
    # print('Counter Textures: x' + str(int_texture_count))
    # print('Counter Faces: x' + str(int_faces_count))

    # Store vertices' co-ordinates
    array_str_X = np.empty(int_vertex_count)
    array_str_Y = np.empty(int_vertex_count)
    array_str_Z = np.empty(int_vertex_count)

    # Store textures' coordinates
    array_str_U = np.empty(int_texture_count)
    array_str_V = np.empty(int_texture_count)

    # Store face indices
    array_str_F = [''] * int_faces_count

    if int_texture_count <= 0:
        print('[----------------- Error -----------------]')
        print('[ OBJ File has no texture information.  ]')
        print('[-----------------------------------------]')

    # Parse co-ordinates and include them in the arrays
    int_vertex_current = 0
    int_texture_current = 0
    int_face_current = 0
    for k in range(len(array_str_obj_encoded)):

        # Split current line by spaces
        str_line = array_str_obj_encoded[k]
        array_str_line = str_line.split()

        # Which is its identifier key?
        str_ident = array_str_line[0]
        if str_ident == 'v':

            # Key 'v' ~ Vertex position, requires X Y Z
            if len(array_str_line) < 4:
                continue

            # Fetch observed X Y Z
            str_x, str_y, str_z = array_str_line[1:4]

            # Add to storage arrays at next index
            array_str_X[int_vertex_current] = str_x
            array_str_Y[int_vertex_current] = str_y
            array_str_Z[int_vertex_current] = str_z
            int_vertex_current += 1

        elif str_ident == 'vt':

            # Key 'vt' ~ Texture Coordinates, requires U V
            if len(array_str_line) < 3:
                continue

            # Fetch observed U V
            str_u, str_v = array_str_line[1:3]

            # Add to storage arrays at next index
            array_str_U[int_texture_current] = str_u
            array_str_V[int_texture_current] = str_v
            int_texture_current += 1

        elif str_ident == 'f':

            # Key 'f' ~ Face information, requires one entry at least
            if len(array_str_line) < 2:
                continue

            # Fetch observed U V
            str_f = str_line[2::]

            # Add to storage arrays at next index
            array_str_F[int_face_current] = str_f
            int_face_current += 1

    # Build maps of Vertex and Textures
    array_int_vector3_ver = toVector3Array(array_str_X, array_str_Y, array_str_Z)
    array_int_vector2_tex = toVector2Array(array_str_U, array_str_V)

    # Create face array and resize
    int_faceAmount = len(array_str_F)
    array_uvFace_faces = [empty_UVFace()] * int_faceAmount

    # Parse and add each face
    int_face_current = 0
    for k in range(int_faceAmount):

        # Current Face?
        str_face_indices = array_str_F[k]

        if int_face_current % 300 == 0:
            print(f'Reading Face #{int_face_current} of #{int_faceAmount}')

        # Parse face
        uvFace_parsed = parse_UVFace(str_face_indices, array_int_vector3_ver, array_int_vector2_tex, -1)

        # Invalid Face ~ less than three vertices
        if len(uvFace_parsed.vertices) < 3:
            print(f"Invalid Face: {str_face_indices}")
            continue

        # Include
        array_uvFace_faces[int_face_current] = uvFace_parsed
        uvFace_parsed.index = int_face_current
        int_face_current += 1

    # Completed (real)
    return UVMap(array_uvFace_faces)

In [ ]:
def betterImRead(str_path, bool_forceRGB=False):

    # Read image and identify specs
    rgbImage_plainTex = cv2.imread(str_path, cv2.IMREAD_UNCHANGED)
    rows = rgbImage_plainTex.shape[0]
    columns = rgbImage_plainTex.shape[1]
    numberOfColorChannels = 1
    if len(rgbImage_plainTex.shape) > 2:
      numberOfColorChannels = rgbImage_plainTex.shape[2]

    if numberOfColorChannels >= 3:

        # It is already RGB
        rgbImage = rgbImage_plainTex

    elif numberOfColorChannels == 1 and len(rgbImage_plainTex.shape) == 2:

        # This is just grayscale
        if bool_forceRGB:

            # Copy that color channel three times
            rgbImage = cv2.merge([rgbImage_plainTex, rgbImage_plainTex, rgbImage_plainTex])
        else:

            # One color channel is good enough
            rgbImage = cv2.cvtColor(rgbImage_plainTex, cv2.COLOR_GRAY2RGB)

    # This file format was agony to debug!
    elif numberOfColorChannels == 1 and len(rgbImage_plainTex.shape) == 3:

        # This is an indexed color map
        int_colorChannels = 1 if not bool_forceRGB else 3
        rgbImage = np.zeros((rows, columns, int_colorChannels), dtype=np.uint8)

        for row in range(rows):
            for col in range(columns):
                val = rgbImage_plainTex[row, col, 0]
                rgbImage[row, col, 0] = cv2.LUT(rgbImage_plainTex, val)

                # If not forcing RGB, only one color channel is needed
                if not bool_forceRGB:
                    continue
                rgbImage[row, col, 1] = cv2.LUT(rgbImage_plainTex, val)
                rgbImage[row, col, 2] = cv2.LUT(rgbImage_plainTex, val)

    return rgbImage

### Drawing onto Texture
These operations allow the rest of the code to edit RGB texture files easily

In [ ]:
def colorMatch(int_r, int_g, int_b, int_val_r, int_val_g, int_val_b, int_colorChannels):

    # No color channels? Technical un-match
    if int_colorChannels < 1:
        return False

    # Check first color channel
    if not np.isnan(int_r):

        # Compare 'at least'
        if int_r < 0 and int_val_r < -int_r:
            return False

        # Compare Equals r
        elif int_val_r != int_r:
            return False

    # Nan, is it present?
    elif int_val_r == 0:
        return False

    # Only one color channel (and it matched?), we are done.
    if int_colorChannels < 2:
        return True

    # Nan, is it present?
    if not np.isnan(int_g):

        # Compare 'at least'
        if int_g < 0 and int_val_g < -int_g:
            return False

        # Compare Equals r
        elif int_val_g != int_g:
            return False

    # Nan, is it present?
    elif int_val_g == 0:
        return False

    # Only two color channel (and they matched?), we are done.
    if int_colorChannels < 3:
        return True

    if not np.isnan(int_b):

        # Compare 'at least'
        if int_b < 0 and int_val_b < -int_b:
            return False

        # Compare Equals r
        elif int_val_b != int_b:
            return False

    # Nan, is it present?
    elif int_val_b == 0:
        return False

    # Reached the end, success
    return True

In [ ]:
def batchRecolor(rgbImage_input, rows, columns, int_tex_r, int_tex_g, int_tex_b, boolean_min, int_r, int_g, int_b):

    #
    #   This function has two modes:
    #
    #   NORMAL mode
    #       Recolors the pixels that match int_r, int_g, int_b
    #       with the color specified int_tex_r, int_tex_g, int_tex_b
    #
    #   MINIMUM mode
    #       Recolors the pixels that match int_r, int_g, int_b
    #       with a gray color, the minimum value of any color channel
    #       of any pixel in the texture, as long as it is not less
    #       than the minimum specifications int_tex_r, int_tex_g, int_tex_b
    #

    if boolean_min:
        int_min = 255

        # Identify the minimum colour?
        for x in range(columns):
            for y in range(rows):

                # Is the colour accepted?
                int_o_r = rgbImage_input[y, x, 0]
                int_o_g = rgbImage_input[y, x, 1]
                int_o_b = rgbImage_input[y, x, 2]

                if int_o_r < int_tex_r or int_o_g < int_tex_g or int_o_b < int_tex_b:
                    continue

                # Minimum of acceptable colours
                if int_o_r < int_min:
                    int_min = int_o_r

                if int_o_g < int_min:
                    int_min = int_o_g

                if int_o_b < int_min:
                    int_min = int_o_b

        # print(f'Minimum Color Identified ~ {int_min}')

        int_tex_r = int_min
        int_tex_g = int_min
        int_tex_b = int_min

    for x in range(columns):
        for y in range(rows):

            int_val_r = rgbImage_input[y, x, 0]
            int_val_g = rgbImage_input[y, x, 1]
            int_val_b = rgbImage_input[y, x, 2]

            # Must match the color channels
            if not colorMatch(int_r, int_g, int_b, int_val_r, int_val_g, int_val_b, 3):
                continue

            # Change color of that pixel
            rgbImage_input[y, x, 0] = int_tex_r
            rgbImage_input[y, x, 1] = int_tex_g
            rgbImage_input[y, x, 2] = int_tex_b

    # Done with these changes
    return rgbImage_input

In [ ]:
def getColor(rgb_image_tex, int_y, int_x, int_channel, int_channels):

    if int_channel > int_channels:
        return 0

    # print(f"At ~{int_x}, {int_y}, {int_channel} found ~{rgb_image_tex[int_y, int_x, int_channel]}~")
    return int(rgb_image_tex[int_y, int_x, int_channel])

In [ ]:
def isPointInFace(vector2_A_png, vector2_B_png, vector2_C_png, x, y):

    # Identify coordinates
    Ux = vector2_A_png.x
    Uy = vector2_A_png.y
    Wx = vector2_B_png.x
    Wy = vector2_B_png.y
    Jx = vector2_C_png.x
    Jy = vector2_C_png.y

    # TOLERANCE
    # ----------------
    Tx = (Ux + Wx + Jx) / 3
    Ty = (Uy + Wy + Jy) / 3
    Kx = ((9 * x) + Tx) / 10
    Ky = ((9 * y) + Ty) / 10

    # UW dot UK
    UWx = Wx - Ux
    UWy = Wy - Uy
    UKx = Kx - Ux
    UKy = Ky - Uy
    UJx = Jx - Ux
    UJy = Jy - Uy
    UW = math.sqrt((UWx * UWx) + (UWy * UWy))
    UK = math.sqrt((UKx * UKx) + (UKy * UKy))
    UJ = math.sqrt((UJx * UJx) + (UJy * UJy))
    angle_UW_UK = math.acos(((UWx * UKx) + (UWy * UKy)) / (UW * UK))
    angle_UW_UJ = math.acos(((UWx * UJx) + (UWy * UJy)) / (UW * UJ))

    # If the angle WUK exceeds WUJ, the point is for sure outside the triangle.
    if angle_UW_UK > angle_UW_UJ:
        return False

    # WJ dot WK
    WJx = Jx - Wx
    WJy = Jy - Wy
    WKx = Kx - Wx
    WKy = Ky - Wy
    WUx = Ux - Wx
    WUy = Uy - Wy
    WJ = math.sqrt((WJx * WJx) + (WJy * WJy))
    WK = math.sqrt((WKx * WKx) + (WKy * WKy))
    WU = math.sqrt((WUx * WUx) + (WUy * WUy))
    angle_WJ_WK = math.acos(((WJx * WKx) + (WJy * WKy)) / (WJ * WK))
    angle_WJ_WU = math.acos(((WJx * WUx) + (WJy * WUy)) / (WJ * WU))

    # If the angle JWK exceeds JWU, the point is for sure outside the triangle.
    if angle_WJ_WK > angle_WJ_WU:
        return False

    # JU dot JK
    JUx = Ux - Jx
    JUy = Uy - Jy
    JKx = Kx - Jx
    JKy = Ky - Jy
    JWx = Wx - Jx
    JWy = Wy - Jy

    # calculating the lengths of the sides JU, JK, and JW
    JU = math.sqrt((JUx * JUx) + (JUy * JUy))
    JK = math.sqrt((JKx * JKx) + (JKy * JKy))
    JW = math.sqrt((JWx * JWx) + (JWy * JWy))

    # calculating the angles between vectors JU and JK, and JU and JW
    angle_JU_JK = math.acos((JUx * JKx + JUy * JKy) / (JU * JK))
    angle_JU_JW = math.acos((JUx * JWx + JUy * JWy) / (JU * JW))

    # checking if the angle UJK exceeds UJW to determine if the point is outside the triangle
    if angle_JU_JK > angle_JU_JW:
        return False

    # Not outside any of the angles, this point is thus inside the triangle
    return True

In [ ]:
def copyTextureOnto(rgbImage_input, rows, columns, rgbImage_tex, double_scale, int_r, int_g, int_b):

    # This will copy the texture of rbgImage_tex onto rbgImage_input,
    # replacing areas where the value r/g/b matches exactly the input.
    #
    # Use NaN as 'any amount of this color' but it must be present
    # Use negative numbers as 'at least' such that -5 means 'at least 5'
    tex_rows, tex_columns, int_colorChannels = rgbImage_tex.shape
    # print(f'IMG {rows}Rx{columns}C')
    # print(f'TEX {tex_rows}Rx{tex_columns}C, {int_colorChannels} Channels')

    # double_c_ratio = tex_columns / columns;
    # double_r_ratio = tex_rows / rows;
    # disp(strcat('RAT ~', strcat(num2str(double_r_ratio), strcat('x', num2str(double_c_ratio)))))
    int_pixels = 0

    for x in range(columns):

        if x % 100 == 0:
            print(f'Scanning row {x}/{columns}')

        for y in range(rows):

            int_val_r = rgbImage_input[y, x, 0]
            int_val_g = rgbImage_input[y, x, 1]
            int_val_b = rgbImage_input[y, x, 2]

            # Must match the color channels
            if not colorMatch(int_r, int_g, int_b, int_val_r, int_val_g, int_val_b, 3):
                continue

            # Must find the corresponding pixels on the 'tex' texture, after scaling it.

            # Shift input-x to tex-x
            double_tex_over_x = x / double_scale
            double_tex_over_y = y / double_scale

            # Strip out the decimal
            double_tex_x_rem = double_tex_over_x - int(double_tex_over_x)
            double_tex_y_rem = double_tex_over_y - int(double_tex_over_y)

            # Modulate over the width of 'tex'
            double_tex_x = np.mod(round(double_tex_over_x - double_tex_x_rem), tex_columns)
            double_tex_y = np.mod(round(double_tex_over_y - double_tex_y_rem), tex_rows)

            # Identify next pixel
            double_tex_x_2 = np.mod(double_tex_x + 1, tex_columns)
            double_tex_y_2 = np.mod(double_tex_y + 1, tex_rows)

            # disp(strcat('PIX 1-1 ~', strcat(num2str(double_tex_x), strcat(',', num2str(double_tex_y)))))
            # disp(strcat('PIX 2-2 ~', strcat(num2str(double_tex_x_2), strcat(',', num2str(double_tex_y_2)))))
            # Calculate weights
            double_wx1 = 1 - double_tex_x_rem
            double_wx2 = double_tex_x_rem
            double_wy1 = 1 - double_tex_y_rem
            double_wy2 = double_tex_y_rem

            # Identify values
            int_r_1_1 = getColor(rgbImage_tex, double_tex_y, double_tex_x, 0, int_colorChannels)
            int_r_2_1 = getColor(rgbImage_tex, double_tex_y, double_tex_x_2, 0, int_colorChannels)
            int_r_1_2 = getColor(rgbImage_tex, double_tex_y_2, double_tex_x, 0, int_colorChannels)
            int_r_2_2 = getColor(rgbImage_tex, double_tex_y_2, double_tex_x_2, 0, int_colorChannels)

            # Weighted sum
            int_r_f = (int_r_1_1 * double_wx1 * double_wy1) + (int_r_2_1 * double_wx2 * double_wy1) + (
                        int_r_1_2 * double_wx1 * double_wy2) + (int_r_2_2 * double_wx2 * double_wy2)

            # Increase pixel
            int_pixels = int_pixels + 1

            # Change color of that yeah
            rgbImage_input[y, x, 0] = int_r_f
            if int_colorChannels < 3:

                # Just gray apparently
                rgbImage_input[y, x, 1] = int_r_f
                rgbImage_input[y, x, 2] = int_r_f
                continue

            int_g_1_1 = getColor(rgbImage_tex, double_tex_y, double_tex_x, 1, int_colorChannels)
            int_g_2_1 = getColor(rgbImage_tex, double_tex_y, double_tex_x_2, 1, int_colorChannels)
            int_g_1_2 = getColor(rgbImage_tex, double_tex_y_2, double_tex_x, 1, int_colorChannels)
            int_g_2_2 = getColor(rgbImage_tex, double_tex_y_2, double_tex_x_2, 1, int_colorChannels)

            # Weighted sum
            int_g_f = (int_g_1_1 * double_wx1 * double_wy1) + (int_g_2_1 * double_wx2 * double_wy1) + (
                        int_g_1_2 * double_wx1 * double_wy2) + (int_g_2_2 * double_wx2 * double_wy2)

            # Change color of that yeah
            rgbImage_input[y, x, 1] = int_g_f

            int_b_1_1 = getColor(rgbImage_tex, double_tex_y, double_tex_x, 2, int_colorChannels)
            int_b_2_1 = getColor(rgbImage_tex, double_tex_y, double_tex_x_2, 2, int_colorChannels)
            int_b_1_2 = getColor(rgbImage_tex, double_tex_y_2, double_tex_x, 2, int_colorChannels)
            int_b_2_2 = getColor(rgbImage_tex, double_tex_y_2, double_tex_x_2, 2, int_colorChannels)

            # Weighted sum
            int_b_f = (int_b_1_1 * double_wx1 * double_wy1) + (int_b_2_1 * double_wx2 * double_wy1) + (
                        int_b_1_2 * double_wx1 * double_wy2) + (int_b_2_2 * double_wx2 * double_wy2)

            # Change color
            rgbImage_input[y, x, 2] = int_b_f

    # Done copying texture
    return [rgbImage_input, int_pixels]

In [ ]:
def fillFace(rgbImage_input, rows, columns, uvFace_textureInfo, int_r, int_g, int_b):
    # Assumes that the uvFace is a TRIANGLE

    # Find UV Location of the Face in the File
    uvVertex_U = uvFace_textureInfo.vertices[0]
    uvVertex_V = uvFace_textureInfo.vertices[1]
    uvVertex_W = uvFace_textureInfo.vertices[2]

    # Find 2D Vertices
    vector2_vtU = uvVertex_U.vt
    vector2_vtV = uvVertex_V.vt
    vector2_vtW = uvVertex_W.vt
    vector2_A_png = Vector2(vector2_vtU.x * columns, (1 - vector2_vtU.y) * rows)
    vector2_B_png = Vector2(vector2_vtV.x * columns, (1 - vector2_vtV.y) * rows)
    vector2_C_png = Vector2(vector2_vtW.x * columns, (1 - vector2_vtW.y) * rows)

    # Identify components
    Ux = vector2_vtU.x
    Uy = vector2_vtU.y
    Vx = vector2_vtV.x
    Vy = vector2_vtV.y
    Wx = vector2_vtW.x
    Wy = vector2_vtW.y

    # Scale to the image sizes
    Ux = Ux * columns
    Vx = Vx * columns
    Wx = Wx * columns

    Uy = (1 - Uy) * rows
    Vy = (1 - Vy) * rows
    Wy = (1 - Wy) * rows

    # Identify order in X coordinate
    if Ux <= Vx:
        if Ux <= Wx:
            if Vx <= Wx:
                # U V W
                Ax = Ux
                Ay = Uy

                Bx = Vx
                By = Vy

                Cx = Wx
                Cy = Wy
            else:
                # U W V
                Ax = Ux
                Ay = Uy

                Bx = Wx
                By = Wy

                Cx = Vx
                Cy = Vy

        # W comes before U
        else:
            # W U V
            Ax = Wx
            Ay = Wy

            Bx = Ux
            By = Uy

            Cx = Vx
            Cy = Vy

    # V comes before U
    else:
        if Vx <= Wx:
            if Ux <= Wx:
                # V U W
                Ax = Vx
                Ay = Vy

                Bx = Ux
                By = Uy

                Cx = Wx
                Cy = Wy
            else:
                # V W U
                Ax = Vx
                Ay = Vy

                Bx = Wx
                By = Wy

                Cx = Ux
                Cy = Uy

        # W comes before V
        else:
            # W V U
            Ax = Wx
            Ay = Wy

            Bx = Vx
            By = Vy

            Cx = Ux
            Cy = Uy

    # Display the coordinates of points A, B and C
    #print(f"A={Ax}, {Ay}; B={Bx}, {By}; C={Cx}, {Cy}")

    # Make sure they are never on the same X
    if Ax == Bx:
        Bx = Ax + 0.00001
    if Ax == Cx:
        Cx = Ax + 0.00002
    if Bx == Cx:
        Cx = Bx + 0.00001

    # Mark a sweet circle
    int_x = round((Ax + Bx + Cx) / 3.00)
    int_y = round((Ay + By + Cy) / 3.00)

    # Gather radius
    double_radius = math.sqrt((int_x - Ax) * (int_x - Ax) + (int_y - Ay) * (int_y - Ay))
    double_radiusB = math.sqrt((int_x - Bx) * (int_x - Bx) + (int_y - By) * (int_y - By))
    double_radiusC = math.sqrt((int_x - Cx) * (int_x - Cx) + (int_y - Cy) * (int_y - Cy))
    if double_radius < double_radiusB:
        double_radius = double_radiusB
    if double_radius < double_radiusC:
        double_radius = double_radiusC

    double_radius = round(double_radius)

    for x in range(int_x - double_radius, int_x + double_radius + 1):
        for y in range(int_y - double_radius, int_y + double_radius + 1):

            # Distance?
            int_xx = x - int_x
            int_yy = y - int_y
            if math.sqrt(int_xx ** 2 + int_yy ** 2) > double_radius:
                # Not drawing a box, drawing a circle!
                continue

            if math.isnan(y) or math.isnan(x) or y < 0 or x < 0 or y >= rows or x >= columns:
                continue

            #         print('- x=' + str(x) + '"')
            #         print('- y=' + str(y) + '"')

            # Same color? I sleep
            if rgbImage_input[y, x, 0] == int_b \
                    and rgbImage_input[y, x, 1] == int_g \
                    and rgbImage_input[y, x, 2] == int_r:
                # No need to color it
                continue

            # In this face?
            if not isPointInFace(vector2_A_png, vector2_B_png, vector2_C_png, x, y):
                continue

            # Change color of that yeah.
            rgbImage_input[y, x, 0] = int_b
            rgbImage_input[y, x, 1] = int_g
            rgbImage_input[y, x, 2] = int_r

    # Complete
    return rgbImage_input

## Generating Mark Texture

In [ ]:
def drawCylinder(array_str_riserPaths, uvMap_map, rgbImage_input, rows, columns, double_tolerance):
    for int_riser in range(len(array_str_riserPaths)):
        print(f'Drawing Riser #{int_riser + 1}')

        # Read Cylinder
        # This method only supports cylindrical risers, but does so at any 3D rotation.
        # It must first identify the axis of the cylinder, it does so by identifying
        # the planes of the faces, both perpendicular to this axis:
        #
        # The geometry of the .STL riser is ultimately given in triangular faces,
        # which in turn defines a finite amount of vertex points.
        #
        # These vertices, they form two circles (defining the top and bottom faces),
        # and the centers of these circles is pierced by one line: The axis of the cylinder.
        #
        # #1 Identify which vertices belong to which plane
        #
        # #2 All the points in a plane make a circle, find its center.
        #
        # #3 There are two planes = two centers. The line they make is the axis of the cylinder.
        #
        # One at this point, the coordinates of the cylinder and its dimensions are readily available.
        #
        # All is left is to treat the cylinder as a set of points, and apply the same algorithm
        # that draws VTK points onto the geometry.

        # Read File
        discreteGeometry_riser = mesh.Mesh.from_file(array_str_riserPaths[int_riser])
        array_vertices = np.unique(discreteGeometry_riser.vectors.reshape((-1, 3)), axis=0)
        int_points = len(array_vertices)

        if int_points < 1:
            # Log error
            print('Empty STL Geometry at ~', array_str_riserPaths[int_riser])
            continue

        # Plane A normal
        PAx = 0
        PAy = 0
        PAz = 0

        # Plane A point is just the very first point in load
        Ax = array_vertices[0, 0]
        Ay = array_vertices[0, 1]
        Az = array_vertices[0, 2]

        # Identify Cylinder, skip the very first point as it is already 'A'
        double_slantTolerance = 0.015
        for i in range(1, int_points - 1):

            # Most points will have their chance to be B
            Bx = array_vertices[i, 0]
            By = array_vertices[i, 1]
            Bz = array_vertices[i, 2]

            # Accept
            bool_accepted = False

            for j in range(i + 1, int_points - 1):

                # Most points will have their chance to be C
                Cx = array_vertices[j, 0]
                Cy = array_vertices[j, 1]
                Cz = array_vertices[j, 2]

                # Plane vector direction yeah
                Px = ((By - Ay) * (Cz - Az)) - ((Bz - Az) * (Cy - Ay))
                Py = ((Bz - Az) * (Cx - Ax)) - ((Bx - Ax) * (Cz - Az))
                Pz = ((Bx - Ax) * (Cy - Ay)) - ((By - Ay) * (Cx - Ax))

                # How many points in plane / how many not
                int_success = 3

                # Do other points lie in this plane?
                for k in range(1, int_points - 1):

                    # Except the two points being evaluated, and the first one k=0 (would be point A)
                    if k == i or k == j:
                        continue

                    # Identify coordinates
                    Vx = array_vertices[k, 0]
                    Vy = array_vertices[k, 1]
                    Vz = array_vertices[k, 2]

                    # Direction within plane
                    AVx = Vx - Ax
                    AVy = Vy - Ay
                    AVz = Vz - Az

                    # Plane equation
                    double_p = (AVx * Px) + (AVy * Py) + (AVz * Pz)

                    # Success?
                    if double_slantTolerance > double_p > -double_slantTolerance:
                        int_success += 1

                    if int_success > (int_points * 0.2):
                        # Accept plane as A
                        bool_accepted = True

                        # Remember plane normal
                        P = ((Px ** 2) + (Py ** 2) + (Pz ** 2)) ** 0.5
                        PAx = Px / P
                        PAy = Py / P
                        PAz = Pz / P
                        break

                if bool_accepted:
                    break

            if bool_accepted:
                break

        # Points on the faces of cylinder
        tot_Ax = 0
        tot_Ay = 0
        tot_Az = 0
        tot_Wx = 0
        tot_Wy = 0
        tot_Wz = 0
        int_Ap = 0
        int_Wp = 0

        # Cylinder Extrema
        double_cylMaxX = Ax
        double_cylMaxY = Ay
        double_cylMaxZ = Az
        double_cylMinX = Ax
        double_cylMinY = Ay
        double_cylMinZ = Az

        for k in range(int_points):

            # Identify coordinates
            Bx = array_vertices[k, 0]
            By = array_vertices[k, 1]
            Bz = array_vertices[k, 2]

            # Extrema Comparison
            if Bx > double_cylMaxX:
                double_cylMaxX = Bx
            if By > double_cylMaxY:
                double_cylMaxY = By
            if Bz > double_cylMaxZ:
                double_cylMaxZ = Bz
            if Bx < double_cylMinX:
                double_cylMinX = Bx
            if By < double_cylMinY:
                double_cylMinY = By
            if Bz < double_cylMinZ:
                double_cylMinZ = Bz

            # Direction within plane
            AVx = Bx - Ax
            AVy = By - Ay
            AVz = Bz - Az

            # Plane equation
            double_p = (AVx * PAx) + (AVy * PAy) + (AVz * PAz)

            # Success?
            if double_p > double_slantTolerance or double_p < -double_slantTolerance:

                # Increase totals
                tot_Wx += Bx
                tot_Wy += By
                tot_Wz += Bz
                int_Wp += 1

            else:

                # Increase totals
                tot_Ax += Bx
                tot_Ay += By
                tot_Az += Bz
                int_Ap += 1

        # Extrema Build, also shift coordinates to UV space
        vector3_uvCylMax = Vector3(double_cylMaxX, double_cylMaxY, double_cylMaxZ).toUVSpace()
        vector3_uvCylMin = Vector3(double_cylMinX, double_cylMinY, double_cylMinZ).toUVSpace()

        # Top Plane Point
        tot_Ax = tot_Ax / int_Ap
        tot_Ay = tot_Ay / int_Ap
        tot_Az = tot_Az / int_Ap
        vector3_Atot = Vector3(tot_Ax, tot_Ay, tot_Az)

        # Bot Plane Point
        tot_Wx = tot_Wx / int_Wp
        tot_Wy = tot_Wy / int_Wp
        tot_Wz = tot_Wz / int_Wp
        vector3_Wtot = Vector3(tot_Wx, tot_Wy, tot_Wz)

        # Also used as radius of cylinder
        double_rad = math.sqrt(((tot_Ax - Ax) ** 2) + ((tot_Ay - Ay) ** 2) + ((tot_Az - Az) ** 2))

        # Find 3D Point to draw at

        # Plane point, origin
        O = vector3_Atot

        # Distance along cylinder
        tAWx = tot_Ax - tot_Wx
        tAWy = tot_Ay - tot_Wy
        tAWz = tot_Az - tot_Wz
        double_OP = math.sqrt((tAWx ** 2) + (tAWy ** 2) + (tAWz ** 2))

        # Plane Unit Vector Direction
        Ox = PAx
        Oy = PAy
        Oz = PAz

        # Another on Plane
        Vx = Ax
        Vy = Ay
        Vz = Az

        # OV directions
        OVx = Vx - Ox
        OVy = Vy - Oy
        OVz = Vz - Oz
        OV = math.sqrt((OVx ** 2) + (OVy ** 2) + (OVz ** 2))
        OVx = OVx / OV
        OVy = OVy / OV
        OVz = OVz / OV

        # Cross Product for the other direction vector
        OWx = (OVy * Oz) - (OVz * Oy)
        OWy = (OVz * Ox) - (OVx * Oz)
        OWz = (OVx * Oy) - (OVy * Ox)

        # Cross Product for correction
        OVx = (Oy * OWz) - (Oz * OWy)
        OVy = (Oz * OWx) - (Ox * OWz)
        OVz = (Ox * OWy) - (Oy * OWx)

        # Encapsulating Box
        array_int_uvFace_boxIntercept = uvMap_map.corneredBoxFaces(vector3_uvCylMin, vector3_uvCylMax, double_tolerance)
        int_mbx = len(array_int_uvFace_boxIntercept)

        # Evaluate every face position in the cylinder
        int_facesDrawn = 0
        for faceIdx in range(int_mbx):

            # Observed face
            uvFace_observed = array_int_uvFace_boxIntercept[faceIdx, 1]
            if not uvFace_observed:
                continue

            # Get vertices
            vector3_A = uvFace_observed.vertices[0].v
            vector3_B = uvFace_observed.vertices[1].v
            vector3_C = uvFace_observed.vertices[2].v

            # (un-convert from UV coordinates)
            vector3_A = vector3_A.toSTLSpace()
            vector3_B = vector3_B.toSTLSpace()
            vector3_C = vector3_C.toSTLSpace()

            # Obtain centroid
            Px = (vector3_A.x + vector3_B.x + vector3_C.x) / 3.0
            Py = (vector3_A.y + vector3_B.y + vector3_C.y) / 3.0
            Pz = (vector3_A.z + vector3_B.z + vector3_C.z) / 3.0

            # Reverse solving of the cylindrical coordinate equations
            #
            #   Note that these cylindrical coordinates are NOW assumed to
            #   be aligned so that the Z axis is the axis of the cylinder,
            #   thus double_z (distance along the axis of the cylinder) is
            #   called 'double_z'
            #
            #   double_Px = (OWx * double_r * cos(double_phi)) + (OVx * double_r * sin(double_phi)) + (Ox * double_z) + O.x;
            #   double_Py = (OWy * double_r * cos(double_phi)) + (OVy * double_r * sin(double_phi)) + (Oy * double_z) + O.y;
            #   double_Pz = (OWz * double_r * cos(double_phi)) + (OVz * double_r * sin(double_phi)) + (Oz * double_z) + O.z;
            #

            # Calculations required for solving for PHI
            double_den = ((Ox * (Py - O.y)) - (Oy * (Px - O.x)))
            double_neg = -1 if double_den < 0 else 1
            double_clef = ((Ox * (Pz - O.z)) - (Oz * (Px - O.x))) / (max(abs(double_den), 1E-7) * double_neg)
            double_gamma = ((Ox * OWz) - (Oz * OWx))
            double_delta = ((Ox * OVz) - (Oz * OVx))
            double_psi = ((Ox * OWy) - (Oy * OWx))
            double_theta = ((Ox * OVy) - (Oy * OVx))

            if Ox == 0:
                Ox = 0.00001
            if Oy == 0:
                Oy = 0.00001
            if Oz == 0:
                Oz = 0.00001

            # Calculate PHI
            double_phi = math.atan(
                - (double_gamma - (double_clef * double_psi)) / (double_delta - (double_clef * double_theta)))

            # Calculate R (now assuming that cylinders are oriented along the Z axis)
            double_r = math.sqrt((Px - O.x) ** 2 + (Py - O.y) ** 2)
            # = ((Ox * (Py - O.y)) - (Oy * (Px - O.x))) / (
            #   (Ox * (OWy * math.cos(double_phi) + OVy * math.sin(double_phi))) - (
            #   Oy * (OWx * math.cos(double_phi) + OVx * math.sin(double_phi))))

            # Calculate Z (now assuming that cylinders are oriented along the Z axis)
            double_z = Pz - O.z  # = ((Px - O.x) - (double_r * (OWx * math.cos(double_phi) + OVx * math.sin(double_phi)))) / Ox

            # print('--| Px =', Px)
            # print('--| Py =', Py)
            # print('--| Pz =', Pz)
            # print('--| r =', double_r)
            # print('--| o =', double_phi)
            # print('--| z =', double_z)

            # Evaluate that the point is inside the cylinder
            # oric_x = (OWx * double_r * math.cos(double_phi)) + (OVx * double_r * math.sin(double_phi)) + (Ox * double_z) + O.x
            # oric_y = (OWy * double_r * math.cos(double_phi)) + (OVy * double_r * math.sin(double_phi)) + (Oy * double_z) + O.y
            # oric_z = (OWz * double_r * math.cos(double_phi)) + (OVz * double_r * math.sin(double_phi)) + (Oz * double_z) + O.z

            # print('--| Px =', Px)
            # print('--| Py =', Py)
            # print('--| Pz =', Pz)
            # print('--| dx =', Px-oric_x)
            # print('--| dy =', Py-oric_y)
            # print('--| dz =', Pz-oric_z)
            # print('--| z =', double_z)
            # print('--| r =', double_r)

            if math.isnan(double_z) or double_z < (double_OP * -0.1) or double_z > double_OP:
                continue

            if math.isnan(double_r) or (double_r * 0.930618) > double_rad or (double_r * 0.930618) < -double_rad:
                # print('Face #', faceIdx, ', R Error, r =', double_r, '/', double_rad)
                continue

            # print('Face #', faceIdx, ', Good, r =', double_r, ', z =', double_z)

            # Color of marking
            int_r = 0
            int_g = 0
            int_b = 255

            # Draw point onto texture
            rgbImage_input = fillFace(rgbImage_input, rows, columns, uvFace_observed, int_r, int_g, int_b)

            int_facesDrawn += 1
            if int_facesDrawn % 300 == 0:
                print(f'Drew face #{uvFace_observed.index} ({faceIdx}/{int_mbx}), total {int_facesDrawn} faces drawn')

    # Result
    return rgbImage_input

## Actually generating the texture PNG file

### Important File Paths

Texture files for each of the regions:

In [ ]:
from google.colab import files
uploaded = files.upload()
str_surface_texture_path = list(uploaded.keys())[0]

Saving A3.png to A3.png


In [ ]:
from google.colab import files
uploaded = files.upload()
str_feeders_texture_path = list(uploaded.keys())[0]

Saving welding.png to welding.png


Result file paths

In [ ]:
# Mark with blue, red, and black colors
str_mark_result_path = 'result_mark.png'

# Actual texture
str_feeders_result_path = 'result_tex.png'

Input File Paths

In [ ]:
# Input uv map file path (.obj)
str_obj_path = '/content/remeshed_uv.obj'

# Input feeder file paths (.stl)
array_str_feeder_paths = np.empty([0])
for i in range(0, num_arrays):
    array_str_feeder_paths = np.append(array_str_feeder_paths, f'/content/feeder{i}.stl')

### More parameters
Generating a 2D texture from features in 3D coordinates is somewhat complicated, so some tolerances must be set.

PNG Texture Result Parameters

In [ ]:
# Dimensions of output texture
int_image_rows = 1024
int_image_columns = 1024

# Create image of the dimensions specified, paint it black in its entirety
rgbImage = np.empty((int_image_rows, int_image_columns, 3), dtype=np.uint8)
for x in range(int_image_columns):
    for y in range(int_image_rows):
        rgbImage[y, x, 0] = 0
        rgbImage[y, x, 1] = 0
        rgbImage[y, x, 2] = 0
cv2.imwrite('/content/unfinished_mark.png', rgbImage)

True

STL Feeder Parameters

In [ ]:
# Feeder Geometry Tolerance
double_feeder_tolerance = 0

# Resolution of shape
double_feeders_resolution = 1

### Mark Geometry
Make markings in a 2D-texture based on 3D features

In [ ]:
#print('---->   Reading OBJ   <----')
uv_map = readOBJFile(str_obj_path)

#print('---->   Drawing Feeders   <----')
rgbImage_mark = drawCylinder(array_str_feeder_paths, uv_map, rgbImage, int_image_rows, int_image_columns, double_feeder_tolerance)

#print('Saving mark result...')
cv2.imwrite(str_mark_result_path, rgbImage_mark)

Reading Face #0 of #189820
Reading Face #300 of #189820
Reading Face #600 of #189820
Reading Face #900 of #189820
Reading Face #1200 of #189820
Reading Face #1500 of #189820
Reading Face #1800 of #189820
Reading Face #2100 of #189820
Reading Face #2400 of #189820
Reading Face #2700 of #189820
Reading Face #3000 of #189820
Reading Face #3300 of #189820
Reading Face #3600 of #189820
Reading Face #3900 of #189820
Reading Face #4200 of #189820
Reading Face #4500 of #189820
Reading Face #4800 of #189820
Reading Face #5100 of #189820
Reading Face #5400 of #189820
Reading Face #5700 of #189820
Reading Face #6000 of #189820
Reading Face #6300 of #189820
Reading Face #6600 of #189820
Reading Face #6900 of #189820
Reading Face #7200 of #189820
Reading Face #7500 of #189820
Reading Face #7800 of #189820
Reading Face #8100 of #189820
Reading Face #8400 of #189820
Reading Face #8700 of #189820
Reading Face #9000 of #189820
Reading Face #9300 of #189820
Reading Face #9600 of #189820
Reading Face #99

True

### Recolor Texture
The new texture has been marked, just copy the pixels off the input texture files onto it, replacing these marks.

In [ ]:
# Resolution of .png textures
double_mark_resolution = uv_map.average_resolution(int_image_rows, int_image_columns, 10000)
print(f'OBJ Resolution = {double_mark_resolution}')

rgbImage_result = rgbImage_mark

# Resolutions (pixels per millimeter)
#
# The Texture Resolution:
#     Resolution of the textures that will be copied onto the result
#
# The Result Resolution:
#     Resolution of the output file from the OBJ calculations that
#     draw in red and blue the parting and feeders of the casting.
double_texScale = 3.3 / math.sqrt(double_mark_resolution)
double_risScale = 0.1 / math.sqrt(double_mark_resolution)

print('---->   Generating Texture   <----')
rgbImage_plainTex = betterImRead(str_surface_texture_path, True)
rgbImage_feederTex = betterImRead(str_feeders_texture_path, True)
print('Recoloring unfinished surface...')
image = cv2.imread('/content/unfinished_mark.png')
# Convert the image to a NumPy ndarray
image_array = np.array(image)
rgbImage_int = copyTextureOnto(np.array(image), int_image_rows, int_image_columns, rgbImage_plainTex, double_texScale, 0, 0, 0)
int_surfacePixel = rgbImage_int[0]
cv2.imwrite('/content/unfinished_texture.png', rgbImage_int[0])

print('Recoloring surface...')
rgbImage_int_generic = copyTextureOnto(rgbImage_result, int_image_rows, int_image_columns, rgbImage_plainTex, double_texScale, 0, 0, 0)
int_surfacePixel = rgbImage_int_generic[1]

print('Recoloring feeders...')
rgbImage_int_feeders = copyTextureOnto(rgbImage_result, int_image_rows, int_image_columns, rgbImage_feederTex, double_risScale, float('nan'), 0, 0)
rgbImage_int_feeders[0] = batchRecolor(rgbImage_int_feeders[0], int_image_rows, int_image_columns, 0, 0, 0, False, 0, 0, float('nan'))
int_feedersPixel = rgbImage_int_feeders[1]

print('Saving feeders texture...')
cv2.imwrite(str_feeders_result_path, rgbImage_int_feeders[0])

OBJ Resolution = 8.331141651479198
---->   Generating Texture   <----
Recoloring unfinished surface...
Scanning row 0/1024
Scanning row 100/1024
Scanning row 200/1024
Scanning row 300/1024
Scanning row 400/1024
Scanning row 500/1024
Scanning row 600/1024
Scanning row 700/1024
Scanning row 800/1024
Scanning row 900/1024
Scanning row 1000/1024
Recoloring surface...
Scanning row 0/1024
Scanning row 100/1024
Scanning row 200/1024
Scanning row 300/1024
Scanning row 400/1024
Scanning row 500/1024
Scanning row 600/1024
Scanning row 700/1024
Scanning row 800/1024
Scanning row 900/1024
Scanning row 1000/1024
Recoloring feeders...
Scanning row 0/1024
Scanning row 100/1024
Scanning row 200/1024
Scanning row 300/1024
Scanning row 400/1024
Scanning row 500/1024
Scanning row 600/1024
Scanning row 700/1024
Scanning row 800/1024
Scanning row 900/1024
Scanning row 1000/1024
Saving feeders texture...


True

### Additional Calculations
Using the number of pixels recolored, and assuming that all faces (and thus pixels) cover a similar area in the texture, one can approximate the area of the actual part that will have to be ground.

Cost of removing these feeders through cutting them and grinding the area

In [ ]:
print('---->   Calculating Cost   <----')

# Write Spreadsheet
rowNames = ['Surface', 'Cutting Feeders', 'Grinding Feeders']
Pixels = [int_surfacePixel, int_feedersPixel, int_feedersPixel]

double_surfaceArea = int_surfacePixel / double_mark_resolution
double_feedersArea = int_feedersPixel / double_mark_resolution
Area_mm2 = [double_surfaceArea, double_feedersArea, double_feedersArea]

double_grindingSpeed = 1           # 1 mm per second
double_cuttingSpeed = 14 / 60     # 14 mm per minute

double_spacing = 1          # #VTK# = vtk_map.spacing.x
double_surfaceTime = 0
double_feedersTime = (2 * math.sqrt(int_feedersPixel) * double_spacing) / (math.sqrt(math.pi * double_mark_resolution) * double_cuttingSpeed * 3600)
double_feedersGrindTime = (2 * math.sqrt(int_feedersPixel) * double_spacing) / (math.sqrt(math.pi * double_mark_resolution) * double_grindingSpeed * 3600)
Time_h = [0, double_feedersTime, double_feedersGrindTime]

double_surfaceCost = 0
double_feederCost = 15          # $15USD per hour
Cost = [double_surfaceCost,
        double_feederCost * double_feedersTime,
        double_feederCost * double_feedersGrindTime]

print(f'Operation:   {rowNames}')
print(f'Area (px):   {Pixels}')
print(f'Area (mm2):  {Area_mm2}')
print(f'Time (h):    {Time_h}')
print(f'Cost ($USD): {Cost}')

---->   Calculating Cost   <----
Operation:   ['Surface', 'Cutting Feeders', 'Grinding Feeders']
Area (px):   [1033917, 14659, 14659]
Area (mm2):  [124102.67923081436, 1759.5427629534167, 1759.5427629534167]
Time (h):    [0, 0.056347634169331445, 0.013147781306177338]
Cost ($USD): [0, 0.8452145125399717, 0.19721671959266007]


# Vizualization Output



### Blender Python code which function included remeshing, UV unwrapping, adding roughness surface

In [ ]:
script = f"""
import bpy

# Delete all objects except for the camera and light
bpy.ops.object.select_all(action='SELECT')
bpy.data.objects['Camera'].select_set(False)
bpy.data.objects['Light'].select_set(False)
bpy.ops.object.delete()

# Import STL mesh and apply remesh modifier
bpy.ops.import_mesh.stl(filepath="/content/{str_filePath}",global_scale=0.1)
bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = {voxel_size}/10
bpy.ops.object.modifier_apply(modifier="Remesh")

# Unwrap the mesh using Smart UV Project
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')

# Add displacement modifier with surface texture
ob = bpy.context.active_object
tex = bpy.data.textures.new(name="surface texture", type="IMAGE")
img = bpy.data.images.load(filepath="/content/unfinished_texture.png")
tex.image = img
tex.extension = 'EXTEND'
mod = ob.modifiers.new("", 'DISPLACE')
mod.texture_coords =  "UV"
mod.strength = 1
mod.mid_level = 0.5
mod.texture = tex
mod = ob.modifiers.get("Displace")
if mod is not None:
    bpy.ops.object.modifier_apply(modifier="Displace")

# Create new material for the mesh and set surface color and metallic value
ob = bpy.context.active_object
mat = bpy.data.materials.new(name="Material")
mat.use_nodes = True
node_tree = mat.node_tree
bsdf_node = node_tree.nodes["Principled BSDF"]
bsdf_node.inputs["Base Color"].default_value = (0.2, 0.2, 0.2, 1.0)
bsdf_node.inputs["Metallic"].default_value = 1
if ob.data.materials:
    ob.data.materials[0] = mat
else:
    ob.data.materials.append(mat)

bpy.ops.object.mode_set(mode="EDIT")
bpy.ops.mesh.subdivide(number_cuts=4)
bpy.ops.object.mode_set(mode="OBJECT")

"""

for i in range(0, num_arrays):
    script += f"""
bpy.ops.import_mesh.stl(filepath="/content/feeder{i}.stl",global_scale=0.1)

bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = {voxel_size}/10
bpy.ops.object.modifier_apply(modifier="Remesh")

# Unwrap the mesh using Smart UV Project
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')

# Add displacement modifier with surface texture
ob = bpy.context.active_object
tex = bpy.data.textures.new(name="surface texture", type="IMAGE")
img = bpy.data.images.load(filepath="/content/unfinished_texture.png")
tex.image = img
tex.extension = 'EXTEND'
mod = ob.modifiers.new("", 'DISPLACE')
mod.texture_coords =  "UV"
mod.strength = 1
mod.mid_level = 0.5
mod.texture = tex
mod = ob.modifiers.get("Displace")
if mod is not None:
    bpy.ops.object.modifier_apply(modifier="Displace")

# Create new material for the mesh and set surface color and metallic value
ob = bpy.context.active_object
mat = bpy.data.materials.new(name="Material{i}")
mat.use_nodes = True
node_tree = mat.node_tree
bsdf_node = node_tree.nodes["Principled BSDF"]
bsdf_node.inputs["Base Color"].default_value = (0.2, 0.2, 0.2, 1.0)
bsdf_node.inputs["Metallic"].default_value = 1
if ob.data.materials:
    ob.data.materials[0] = mat
else:
    ob.data.materials.append(mat)

bpy.ops.object.mode_set(mode="EDIT")
bpy.ops.mesh.subdivide(number_cuts=4)
bpy.ops.object.mode_set(mode="OBJECT")
"""
script +="""
bpy.ops.export_scene.gltf(filepath="/content/unfinished_geometry.glb")
"""

# Save the script to a text file
with open('unfinished_geometry.py', 'w') as f:
    f.write(script)

In [ ]:
!./$blender_path -b  -noaudio -P "/content/unfinished_geometry.py"

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Import finished in 0.0044 sec.
Import finished in 0.0026 sec.
03:42:28 | INFO: Draco mesh compression is available, use library at /content/blender-3.4.1-linux-x64/3.4/python/lib/python3.10/site-packages/libextern_draco.so
03:42:28 | INFO: Starting glTF 2.0 export
03:42:32 | INFO: Extracting primitive: SA30053 3D MODEL (2)
03:43:11 | INFO: Primitives created: 1
03:43:13 | INFO: Extracting primitive: feeder0
03:43:20 | INFO: Primitives created: 1
03:43:26 | INFO: Finished glTF 2.0 export in 57.59981083869934 s


Blender quit


In [ ]:
script = f"""
import bpy
# Delete all objects except for the camera and light
bpy.ops.object.select_all(action='SELECT')
bpy.data.objects['Camera'].select_set(False)
bpy.data.objects['Light'].select_set(False)
bpy.ops.object.delete()

# Import STL mesh and apply remesh modifier
bpy.ops.import_mesh.stl(filepath="/content/{str_filePath}",global_scale=0.1)
bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = {voxel_size}/10
bpy.ops.object.modifier_apply(modifier="Remesh")

# Unwrap the mesh using Smart UV Project
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')
# Create a new material
material = bpy.data.materials.new(name="MyMaterial")

# Create a new shader node tree for the material
material.use_nodes = True
tree = material.node_tree
nodes = tree.nodes

# Clear existing nodes
for node in nodes:
    nodes.remove(node)

# Create a new Principled BSDF node
bsdf_node = nodes.new(type='ShaderNodeBsdfPrincipled')
bsdf_node.location = (0, 0)  # Position the node in the node editor

# Create a new Image Texture node
texture_node = nodes.new(type='ShaderNodeTexImage')
texture_node.location = (-200, 0)  # Position the node in the node editor

# Set the path to the image texture
texture_node.image = bpy.data.images.load("/content/result_mark.png")

# Connect the Image Texture node to the Base Color input of the Principled BSDF node
base_color_link = tree.links.new(texture_node.outputs['Color'], bsdf_node.inputs['Base Color'])

# Create a new Material Output node
output_node = nodes.new(type='ShaderNodeOutputMaterial')
output_node.location = (200, 0)  # Position the node in the node editor

# Connect the Principled BSDF node to the Surface input of the Material Output node
surface_link = tree.links.new(bsdf_node.outputs['BSDF'], output_node.inputs['Surface'])

# Assign the material to the active object
bpy.context.object.data.materials.append(material)
"""
script +="""
bpy.ops.export_scene.gltf(filepath="/content/colored_finished_geometry.glb")
"""

# Save the script to a text file
with open('colored_finished_geometry.py', 'w') as f:
    f.write(script)

In [ ]:
!./$blender_path -b  -noaudio -P "/content/colored_finished_geometry.py"

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Import finished in 0.0038 sec.
02:57:07 | INFO: Draco mesh compression is available, use library at /content/blender-3.4.1-linux-x64/3.4/python/lib/python3.10/site-packages/libextern_draco.so
02:57:07 | INFO: Starting glTF 2.0 export
02:57:07 | INFO: Extracting primitive: SA30053 3D MODEL (2)
02:57:08 | INFO: Primitives created: 1
02:57:08 | INFO: Finished glTF 2.0 export in 1.0693578720092773 s


Blender quit


In [ ]:
script = f"""
import bpy

# Delete all objects except for the camera and light
bpy.ops.object.select_all(action='SELECT')
bpy.data.objects['Camera'].select_set(False)
bpy.data.objects['Light'].select_set(False)
bpy.ops.object.delete()

# Import STL mesh and apply remesh modifier
bpy.ops.import_mesh.stl(filepath="/content/{str_filePath}",global_scale=0.1)
bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = {voxel_size}/10
bpy.ops.object.modifier_apply(modifier="Remesh")

# Unwrap the mesh using Smart UV Project
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')

# Add displacement modifier with surface texture
ob = bpy.context.active_object
tex = bpy.data.textures.new(name="surface texture", type="IMAGE")
img = bpy.data.images.load(filepath="/content/result_tex.png")
tex.image = img
tex.extension = 'EXTEND'
mod = ob.modifiers.new("", 'DISPLACE')
mod.texture_coords =  "UV"
mod.strength = 1
mod.mid_level = 0.5
mod.texture = tex
mod = ob.modifiers.get("Displace")
if mod is not None:
    bpy.ops.object.modifier_apply(modifier="Displace")

# Create new material for the mesh and set surface color and metallic value
ob = bpy.context.active_object
mat = bpy.data.materials.new(name="Material")
mat.use_nodes = True
node_tree = mat.node_tree
bsdf_node = node_tree.nodes["Principled BSDF"]
bsdf_node.inputs["Base Color"].default_value = (0.2, 0.2, 0.2, 1.0)
bsdf_node.inputs["Metallic"].default_value = 1
if ob.data.materials:
    ob.data.materials[0] = mat
else:
    ob.data.materials.append(mat)

bpy.ops.object.mode_set(mode="EDIT")
bpy.ops.mesh.subdivide(number_cuts=4)
bpy.ops.object.mode_set(mode="OBJECT")

"""
script +="""
bpy.ops.export_scene.gltf(filepath="/content/textured_finished_geometry.glb")
"""

# Save the script to a text file
with open('textured_finished_geometry.py', 'w') as f:
    f.write(script)

In [ ]:
!./$blender_path -b  -noaudio -P "/content/textured_finished_geometry.py"

Blender 3.4.1 (hash 55485cb379f7 built 2022-12-20 00:46:45)
Import finished in 0.0040 sec.
03:13:49 | INFO: Draco mesh compression is available, use library at /content/blender-3.4.1-linux-x64/3.4/python/lib/python3.10/site-packages/libextern_draco.so
03:13:50 | INFO: Starting glTF 2.0 export
03:13:55 | INFO: Extracting primitive: SA30053 3D MODEL (2)
03:14:33 | INFO: Primitives created: 1
03:14:40 | INFO: Finished glTF 2.0 export in 49.95235061645508 s


Blender quit
